In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
import pandas as pd
import os
from google.colab import drive
from collections import defaultdict
glove_path = '/content/drive/MyDrive/word_embeddings/glove/glove.6B.300d.txt'
def load_glove_embeddings(glove_path):
    embeddings = {}
    with open(glove_path, 'r', encoding='utf-8') as file:
        for line in file:
            values = line.split()
            word = values[0]
            vector = np.asarray(values[1:], dtype='float32')
            embeddings[word] = vector
    return embeddings

glove_embeddings = load_glove_embeddings(glove_path)
print(f"Loaded {len(glove_embeddings)} word vectors.")
def normalize_embeddings(embeddings):
    norm_embeddings = {}
    for word, vec in embeddings.items():
        norm_embeddings[word] = vec / np.linalg.norm(vec)
    return norm_embeddings

# Normalize GloVe embeddings
normalized_glove_embeddings = normalize_embeddings(glove_embeddings)
occupation_words = []
with open('words.txt','r') as file:
  occupation_words = [line.rstrip().replace(' ','-').lower() for line in file]
def check_oov_words(word_list, embeddings):
    oov_words = []
    for word in word_list:
        if word not in embeddings:
            oov_words.append(word)
    return oov_words

oov_gendered = check_oov_words(gendered_words, normalized_glove_embeddings)
oov_occupation = check_oov_words(occupation_words, normalized_glove_embeddings)

print(f"OOV gendered words: {oov_gendered}")
print(f"OOV occupation words: {oov_occupation}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loaded 400000 word vectors.
OOV gendered words: []
OOV occupation words: ['', '', '', '', '', '', 'flight-attendant', '', 'social-worker', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'police-officer', '', 'security-guard', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'data-scientist', '', 'data-analyst', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', 'office-assistant', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '', '']


In [ ]:
occupation_words =set(occupation_words)
oov_occupation =  set(oov_occupation)
occupation_words = occupation_words - oov_occupation

In [ ]:
len(occupation_words)

112

In [ ]:
from sklearn.decomposition import PCA
def pca(matrix):
  n_components = 10
  pca = PCA(n_components=n_components)
  principal_components = pca.fit_transform(matrix)
  principal_component_vector = pca.components_[0]
  #principal_df = pd.DataFrame(principal_components, columns=[f"PC{i+1}" for i in range(n_components)])
  print("Explained variance ratio of each component:", pca.explained_variance_ratio_)
  #print("Principal Components:\n", principal_df)

  return principal_component_vector

gender_pairs = [('he', 'she'),('his','her'), ('man', 'woman'),('john','mary'),('boy', 'girl'),('himself','herself'),('son','daughter'),('guy','gal'), ('father', 'mother'), ('male','female') ]
matrix =[]
for a,b in gender_pairs:
  matrix.append(normalized_glove_embeddings[b] - normalized_glove_embeddings[a])
matrix = np.array(matrix)
df = pca(matrix)
gender_axis = df
from scipy.spatial.distance import cosine
def cosine_similarity(vec1, vec2):
    """Compute cosine similarity between two vectors."""
    return 1 - cosine(vec1, vec2)
def compute_gender_bias(embeddings, occupation_words, gender_axis):
    bias_scores = {}
    for word in occupation_words:
            if word not in embeddings:
              print('here')
            similarity = cosine_similarity(embeddings[word], gender_axis)
            bias_scores[word] = similarity
    return bias_scores
scores = compute_gender_bias(normalized_glove_embeddings,occupation_words,gender_axis)
final_score =0
for key in scores.keys():
  final_score+=abs(scores[key])
final_score/= len(scores)
final_score

Explained variance ratio of each component: [4.0312684e-01 2.1468970e-01 1.4305896e-01 7.0180491e-02 6.0047414e-02
 4.5551356e-02 3.2000728e-02 1.9898888e-02 1.1445663e-02 5.9663950e-15]


0.09191059272812405

In [ ]:
# Phase 3: Hard Debiasing

import numpy as np
from sklearn.decomposition import PCA

# Step 1: Define gendered words for subspace calculation
gendered_words = ['he', 'she', 'man', 'woman', 'father', 'mother', 'husband', 'wife', 'boy', 'girl']
gendered_embeddings = [normalized_glove_embeddings[word] for word in gendered_words if word in normalized_glove_embeddings]

# Step 2: Compute the gender direction (subspace)
# We use PCA (Principal Component Analysis) to find the direction of gender bias
def compute_gender_subspace(embeddings):
    embeddings_matrix = np.array(embeddings)
    pca = PCA(n_components=1)
    pca.fit(embeddings_matrix)
    return pca.components_[0]

gender_subspace = gender_axis

# Step 3: Project embeddings onto the subspace and remove gender component
def remove_gender_bias(embeddings, gender_subspace):
    debiased_embeddings = {}
    for word, vector in embeddings.items():
        projection = np.dot(vector, gender_subspace) * gender_subspace
        debiased_vector = vector - projection  # Remove the gendered component
        debiased_embeddings[word] = debiased_vector
    return debiased_embeddings

embeddings = {word:normalized_glove_embeddings[word] for word in occupation_words}
# Apply hard debiasing to the embeddings
debiased_embeddings = remove_gender_bias(embeddings, gender_subspace)

# Step 4: Save debiased embeddings for future use
#np.save('/content/drive/MyDrive/word_embeddings/debiased_glove_embeddings.npy', debiased_embeddings)
print("Debiased embeddings saved.")

# Step 5: Evaluate the debiased embeddings (recompute cosine similarity and WEAT score)

# Compute cosine similarities for gendered word pairs after debiasing
debiased_gender_bias_scores = compute_gender_bias(debiased_embeddings,occupation_words, gender_subspace)

# Recompute WEAT score after debiasing
#debiased_weat_score = weat_score(debiased_embeddings, target_words, attribute_words)

# Display results after debiasing
#print("\nDebiased Gender Bias Cosine Similarities:")
#for pair, score in debiased_gender_bias_scores.items():
#    print(f"Similarity between {pair[0]} and {pair[1]}: {score:.4f}")

#print(f"\nDebiased WEAT Score for gender bias in occupations: {debiased_weat_score:.4f}")
sum(abs(number) for word,number in debiased_gender_bias_scores.items())/len(debiased_gender_bias_scores)




Debiased embeddings saved.


1.1931927629624715e-08

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from gensim.models import KeyedVectors
from itertools import islice
from torch.utils.data import Dataset, DataLoader
# Load pre-trained GloVe embeddings (fixed embeddings)
def load_glove_embeddings(file_path):
    glove_model = KeyedVectors.load_word2vec_format(file_path, binary=False)
    return glove_model

# Example: Load GloVe 300d embeddings

glove_model = dict(islice(normalized_glove_embeddings.items(), 10000))

# Function to get embedding for a word
def get_embedding(word):
    if word in glove_model:
        return glove_model[word]
    else:
        return np.zeros(glove_model.vector_size)

# Embedding Model (fixed pre-trained embeddings) with additional layers for feature extraction
class FixedEmbeddingModel(nn.Module):
    def __init__(self, embedding_dim=300):
        super(FixedEmbeddingModel, self).__init__()
        self.fc1 = nn.Linear(embedding_dim, 512)
        self.dropout1 = nn.Dropout(p=0.3)
        self.fc2 = nn.Linear(512, 512)
        self.dropout2 = nn.Dropout(p=0.3)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, embedding_dim)

    def forward(self, word_embeddings):
        x = word_embeddings
        x = F.relu(self.fc1(x))
        x = self.dropout1(x)
        x = F.relu(self.fc2(x))
        x = self.dropout2(x)
        x = F.relu(self.fc3(x))
        x = self.fc4(x) + word_embeddings  # Residual connection
        return x

In [ ]:
class GloveDataset(Dataset):
    def __init__(self, glove_model):
        self.glove_model = glove_model
        self.words = list(glove_model.keys())

    def __len__(self):
        return len(self.words)

    def __getitem__(self, idx):
        word = self.words[idx]
        embedding = torch.tensor(self.glove_model[word], dtype=torch.float32)
        return embedding

In [ ]:
def train_embedding_autoencoder(model, glove_model, batch_size=32, epochs=100, lr=0.001):
    dataset = GloveDataset(glove_model)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()

    for epoch in range(epochs):
        model.train()
        epoch_loss = 0

        for batch in dataloader:
            # Forward pass
            output = model(batch)
            loss = criterion(output, batch)  # Compute reconstruction loss
            epoch_loss += loss.item() * batch.size(0)  # Accumulate loss for the batch

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        # Average loss over the entire dataset
        print(f"Epoch {epoch + 1}/{epochs}, Loss: {epoch_loss / len(dataset)}")

In [ ]:
model = FixedEmbeddingModel(embedding_dim=300)

train_embedding_autoencoder(model, glove_model, batch_size=32, epochs=20, lr=0.0001)

Epoch 1/20, Loss: 0.00012167827507655603
Epoch 2/20, Loss: 8.112582772446331e-06
Epoch 3/20, Loss: 4.379992613394279e-06
Epoch 4/20, Loss: 2.759294563293224e-06
Epoch 5/20, Loss: 1.772112929575087e-06
Epoch 6/20, Loss: 1.0776877008538576e-06
Epoch 7/20, Loss: 5.955111997536733e-07
Epoch 8/20, Loss: 3.102728729800219e-07
Epoch 9/20, Loss: 1.6353344874460162e-07
Epoch 10/20, Loss: 8.566146744328761e-08
Epoch 11/20, Loss: 5.093084825489314e-08
Epoch 12/20, Loss: 3.1088684090718744e-08
Epoch 13/20, Loss: 1.9271906919016145e-08
Epoch 14/20, Loss: 1.3324629794198018e-08
Epoch 15/20, Loss: 9.023412275865894e-09
Epoch 16/20, Loss: 6.336285468933056e-09
Epoch 17/20, Loss: 4.9383170320993486e-09
Epoch 18/20, Loss: 3.7127947717152664e-09
Epoch 19/20, Loss: 2.8136686371027507e-09
Epoch 20/20, Loss: 2.151436447128674e-09


In [ ]:
glove_model['he'].mean()

-0.00116658

In [ ]:
def evaluate_embedding_autoencoder(model, glove_model, batch_size=32):
    model.eval()  # Set the model to evaluation mode
    dataset = GloveDataset(glove_model)
    dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=False)
    criterion = nn.MSELoss()

    total_loss = 0

    with torch.no_grad():  # Disable gradient calculation for evaluation
        for batch in dataloader:
            # Forward pass
            output = model(batch)
            loss = criterion(output, batch)  # Compute reconstruction loss
            total_loss += loss.item() * batch.size(0)  # Accumulate loss for the batch

    # Average loss over the entire dataset
    avg_loss = total_loss / len(dataset)
    print(f"Evaluation Loss (Reconstruction Error): {avg_loss}")
    return avg_loss

In [ ]:
print(evaluate_embedding_autoencoder(model,glove_model))

Evaluation Loss (Reconstruction Error): 3.512577190001309e-10
3.512577190001309e-10


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Gradient Reversal Layer (GRL)
class GradientReversalLayer(torch.autograd.Function):
    @staticmethod
    def forward(ctx, x, alpha=1.0):
        ctx.alpha = alpha
        return x.clone()

    @staticmethod
    def backward(ctx, grad_output):
        return -ctx.alpha * grad_output, None

# Adversary Model (predicts sensitive attribute, e.g., gender)
class AdversaryModel(nn.Module):
    def __init__(self, input_dim):
        super(AdversaryModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, 128)
        # self.fc2 = nn.Linear(128, 256)
        # self.fc3 = nn.Linear(256, 128)
        self.fc4 = nn.Linear(128, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        # x = torch.relu(self.fc2(x))
        # x = torch.relu(self.fc3(x))
        x = self.fc4(x)
        return self.sigmoid(x)

# Function to apply GRL
def apply_grl(x, alpha=1.0):
    return GradientReversalLayer.apply(x, alpha)

# Updated Training loop for adversarial debiasing
def train_adversarial_model(glove_model,adversary_model, embedding_model, num_epochs=100, lr=0.001, alpha=1.0):
    # Initialize adversary model
      # Use the dimension of GloVe embeddings
    optimizer = optim.Adam(list(embedding_model.parameters()) + list(adversary_model.parameters()), lr=lr)
    criterion = nn.BCELoss()

    # Example words and gender labels (1 for male, 0 for female)
    words = scores.keys()
    labels = torch.tensor([scores[x] for x in scores.keys() ], dtype=torch.float32)  # Gender labels
    labels = torch.sigmoid(labels)
    # min_score =labels.min()
    # max_score = labels.max()
    # labels = (labels - min_score) / (max_score - min_score)
    for epoch in range(num_epochs):
        embedding_model.train()
        adversary_model.train()
        optimizer.zero_grad()

        # Get embeddings from glove_model, then pass through the embedding model
        embeddings = torch.stack([torch.tensor(glove_model[word], dtype=torch.float32) for word in words])
        transformed_embeddings = embedding_model(embeddings)  # Pass through embedding model

        # Apply the Gradient Reversal Layer (GRL)
        reversed_embeddings = apply_grl(transformed_embeddings, alpha)

        # Adversary tries to predict gender from the embeddings
        gender_predictions = adversary_model(reversed_embeddings).squeeze()

        # Calculate adversarial loss
        adversary_loss = criterion(gender_predictions, labels)

        # Backpropagate and update the model
        adversary_loss.backward()
        optimizer.step()

        # Print the loss every 10 epochs
        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch+1}/{num_epochs}], Adversarial Loss: {adversary_loss.item():.4f}")

# Example usage



In [ ]:
import copy
model_use = copy.deepcopy(model)

In [ ]:
adversary_model = AdversaryModel(input_dim=300)
train_adversarial_model(normalized_glove_embeddings,adversary_model, model_use, num_epochs=100, lr=0.0009, alpha=3.5)

Epoch [10/100], Adversarial Loss: 0.6938
Epoch [20/100], Adversarial Loss: 0.7019
Epoch [30/100], Adversarial Loss: 0.6999
Epoch [40/100], Adversarial Loss: 0.6960
Epoch [50/100], Adversarial Loss: 0.6940
Epoch [60/100], Adversarial Loss: 0.6978
Epoch [70/100], Adversarial Loss: 0.7177
Epoch [80/100], Adversarial Loss: 0.8562
Epoch [90/100], Adversarial Loss: 0.9549
Epoch [100/100], Adversarial Loss: 1.0867


In [ ]:
def get_new_embeddings(word_embedding_dict, embedding_model):
    # List to store the updated embeddings
    updated_embeddings = {}

    # Set the model to evaluation mode (since we're not training now)
    embedding_model.eval()

    with torch.no_grad():  # Disable gradient calculation
        for word, embedding in word_embedding_dict.items():
            # Convert the embedding to a torch tensor
            embedding_tensor = torch.tensor(embedding, dtype=torch.float32).unsqueeze(0)  # Add batch dimension

            # Forward pass through the trained model
            updated_embedding = embedding_model(embedding_tensor)  # Model expects a batch, so pass as tensor

            # Store the updated embedding back into the dictionary
            updated_embeddings[word] = updated_embedding.squeeze(0).numpy()  # Convert tensor to numpy array and remove batch dimension

    return updated_embeddings


In [ ]:
oc_embeddings = {word:normalized_glove_embeddings[word] for word in occupation_words}

In [ ]:
updated_embeddings = get_new_embeddings(oc_embeddings, model_use)

In [ ]:
new_scores = compute_gender_bias(updated_embeddings,occupation_words,gender_axis)
final_score =0
for key in new_scores.keys():
  final_score+=abs(new_scores[key])
final_score/= len(new_scores)
final_score

0.055273557817908214

In [ ]:
import pickle

with open('model.pkl', 'wb') as file:
    pickle.dump(model_use, file)

In [ ]:
with open('embeddings.pkl', 'wb') as file:
    pickle.dump(updated_embeddings,file)

In [ ]:
final_embeddings = copy.deepcopy(normalized_glove_embeddings)
for word in updated_embeddings.keys():
  final_embeddings[word] = updated_embeddings[word]

In [ ]:
import numpy as np

def calculate_weat(embeddings, target_set, attribute_sets):
    # Define gendered attribute sets
    attribute_A, attribute_B = attribute_sets

    # Helper function to calculate average cosine similarity of target to an attribute set
    def association(target_vector, attribute_set_vectors):
        return np.mean([np.dot(target_vector, attr_vec) for attr_vec in attribute_set_vectors])

    # Calculate WEAT score
    weat_score = 0
    attribute_A_vectors = [embeddings.get(word, np.zeros(300)) for word in attribute_A]
    attribute_B_vectors = [embeddings.get(word, np.zeros(300)) for word in attribute_B]

    for word in target_set:
        word_vec = embeddings.get(word, np.zeros(300))
        weat_score += association(word_vec, attribute_A_vectors) - association(word_vec, attribute_B_vectors)

    return weat_score

# Example occupation words list
occupation_words = [word for word in updated_embeddings]

# Define gendered word lists
gendered_words_male = ["he", "him", "his", "man", "male"]
gendered_words_female = ["she", "her", "hers", "woman", "female"]

# Define the target and attribute sets
target_set = occupation_words
attribute_sets = [gendered_words_male, gendered_words_female]

# Calculate the WEAT score
weat_score = calculate_weat(updated_embeddings, target_set, attribute_sets)
print("WEAT Score:", weat_score)


WEAT Score: 0.0


In [ ]:
weat_score = calculate_weat(normalized_glove_embeddings, target_set, attribute_sets)
print("WEAT Score:", weat_score)

WEAT Score: -0.5812739068642259


In [ ]:
weat_score = calculate_weat(debiased_embeddings, target_set, attribute_sets)
print("WEAT Score:", weat_score)

WEAT Score: 0.0
